In [1]:
# source: https://www.kaggle.com/nobletp/panda-keras-baseline

In [2]:
#test: python3 effnB3kappa_da_wb.py --cnnpar effnB3_test --mfolder effnB2kappa_deepaug
#train: python3 effnB2kappa_da_wb.py --cnnpar effnB3_da --mfolder effnB3kappa_da_wb

In [1]:
import warnings
warnings.filterwarnings('ignore')
import os, argparse, sys
sys.path.append("..")
from glob import glob
from random import shuffle
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd 
import json
import skimage.io
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
import efficientnet.tfkeras as efn
print('tensorflow version:', tf.__version__)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
else:
    print('no gpus')

tensorflow version: 2.2.0
no gpus


In [2]:
#import albumentations as albu
# from imgaug import augmenters as iaa
# import imgaug as ia
# ia.seed(1)

In [3]:
# from imgaug import augmenters as iaa
# sometimes = lambda aug: iaa.Sometimes(0.5, aug)

In [5]:
import panda_bvv_config
from panda_bvv_config import *

In [7]:
# #FOR TRAIN:
# ap = argparse.ArgumentParser()
# ap.add_argument('--cnnpar', type=str, help="parameters name", dest = 'cnn_parameters', default = 'effnB2')
# ap.add_argument('--mfolder', help="folder to save model files", dest = 'mfolder', default = 'effnB2_model',\
#                 type=str)
#                 #("--cnn", type=str, help="training cnn name", dest = 'train_cnn')
# args = vars(ap.parse_args())
# cnnet = args["cnn_parameters"]
# model_save_folder = args["mfolder"]


In [6]:
#FOR TEST:
cnnet = 'effnB3_test'
model_save_folder = 'effnB3kappa'

In [7]:
full_model_save_folder_path = os.path.join(note_path, model_save_folder)
input_parameters = panda_bvv_config.train_dict.get(cnnet)

image_sizey = input_parameters.get('image_sizey')
image_sizex = input_parameters.get('image_sizex')
num_epochs = input_parameters.get('num_epochs')
num_reduceOnPlateu = input_parameters.get('num_reduceOnPlateu')
learn_rate = input_parameters.get('learn_rate')
stop_patience = input_parameters.get('stop_patience')
inp_label_smooth = input_parameters.get('inp_label_smooth')
our_id_label_map = input_parameters.get('id_label_map')
class_weights_ = input_parameters.get('class_weights')
output_bias = tf.keras.initializers.Constant(input_parameters.get('output_bias'))
BS = input_parameters.get('BS')
s_per_epoch = input_parameters.get('s_per_epoch')
val_steps = input_parameters.get('val_steps')
model_name = input_parameters.get('model_name')
checkpoint_name = input_parameters.get('checkpoint_name')
weights_file = input_parameters.get('weights_file')
file_for_struct = input_parameters.get('file_for_struct')
file_for_weights = input_parameters.get('file_for_weights')
history_file = input_parameters.get('history_file')
save_plot_file = input_parameters.get('save_plot_file')
num_logits = input_parameters.get('num_logits')
from_folder_train = os.path.join(base_path, input_parameters.get('from_folder_train'))
from_folder_val = os.path.join(base_path, input_parameters.get('from_folder_val'))
if input_parameters.get('bestmodel_weights'): bestmodel_weights = input_parameters.get('bestmodel_weights')
input_shape_ =(image_sizey, image_sizex , 3)
TrDataGen = input_parameters.get('trdatagen') 
ValDataGen = input_parameters.get('valdatagen') 


In [8]:
from tensorflow_addons.metrics import  CohenKappa
kappa_keras = CohenKappa(num_classes=num_logits,weightage='quadratic')

In [9]:
train, val = train_test_split(glob(from_folder_train +'/*.png'), 
                              test_size= val_size_proportion, 
                              random_state=random_state_split)

if from_folder_val: 
    _, val = train_test_split(glob(from_folder_val +'/*.png'), 
                              test_size= val_size_proportion, 
                              random_state=random_state_split)
    print(f"[INFO:] validation samples from val folder {input_parameters.get('from_folder_val')}")
else:
    print("[INFO:] validation samples from train folder")

[INFO:] validation samples from val folder gs2_16x320


In [13]:
#full_model_save_folder_path = os.path.join(note_path, model_save_folder)
if not os.path.exists(full_model_save_folder_path):
    print("[INFO] 'creating {}' directory".format(model_save_folder))
    os.makedirs(full_model_save_folder_path)
model_name = os.path.join(full_model_save_folder_path, model_name)
checkpoint_name = os.path.join(full_model_save_folder_path, checkpoint_name)

weights_file = os.path.join(note_path, weights_file) #!:not the same path

file_for_struct = os.path.join(full_model_save_folder_path, file_for_struct)
file_for_weights = os.path.join(full_model_save_folder_path, file_for_weights)
history_file = os.path.join(full_model_save_folder_path, history_file)
save_plot_file_main = os.path.join(full_model_save_folder_path, 'acc_' + save_plot_file)
save_plot_file_kappa = os.path.join(full_model_save_folder_path, 'kappa_' + save_plot_file)

In [10]:
print('CNN input parameters:\n') 
for k, v in input_parameters.items():
    if k != 'id_label_map':
        print('{}: {}'.format(k, v))

CNN input parameters:

image_sizey: 320
image_sizex: 320
num_epochs: 2
num_earlyStop: 2
num_reduceOnPlateu: 8
learn_rate: 0.0005
stop_patience: 14
inp_label_smooth: 0.01
BS: 10
s_per_epoch: 20
val_steps: 8
class_weights: {0: 3.67082, 1: 3.982, 2: 7.90469, 3: 8.5475, 4: 8.4996, 5: 8.6732}
output_bias: [2.448 2.367 1.681 1.603 1.608 1.588]
model_name: model_panda.h5
checkpoint_name: model_effnB3_panda_check
weights_file: efficientnet-b3_weights_tf_dim_ordering_tf_kernels_autoaugment_notop.h5
bestmodel_weights: best12_weights.h5
level0_file: None
file_for_struct: model_effnB3_panda_struct.json
file_for_weights: model_effnB3_panda_weights.json
history_file: history_effnB3.json
save_plot_file: plot_edu_effnb3.png
from_folder_train: gs2_16x320
from_folder_val: gs2_16x320
num_logits: 6
trdatagen: <function MultiClass.<locals>.wrapper at 0x7f656ea80d90>
valdatagen: <function MultiClass.<locals>.wrapper at 0x7f656ea80d90>


In [ ]:
with open(os.path.join(full_model_save_folder_path, 'parameters_' + model_name.split('.')[0]), 'w') as ff:
    for k, v in input_parameters.items():
        if k != 'id_label_map':
            ff.write(f'{k}:{v}\n')

In [14]:
# train_datagen = TrDataGen(
#         train_cnn,
#         image_sizey, image_sizex,
#         batch_size_= BS,
#         shuffle_=True)
# val_datagen = ValDataGen(
#         valid_cnn,
#         image_sizey, 
#         image_sizex,
#         batch_size_ = BS,
#         shuffle_=False)

In [11]:
train_datagen = TrDataGen(list_files = train,
                    id_label_map = our_id_label_map, 
                    batch_size = BS,
                    depth = num_logits,
                    augment=True,
                    shuf = False)
val_datagen = ValDataGen(list_files = val, 
                    id_label_map = our_id_label_map, 
                    batch_size = BS, 
                    depth = num_logits,
                    augment=False,
                    shuf = False)

In [12]:
#skimage.io.imshow(tt[0][6])

In [13]:
callbacks_list = [
#         EarlyStopping(
        # Stop training when `val_loss` is no longer improving
#         monitor='val_loss',
#         mode = 'min',
#         min_delta=1e-2,
#         patience=stop_patience,
#         verbose=1,
#         restore_best_weights = True
#         ),
        
        ModelCheckpoint(
        filepath= checkpoint_name +".{epoch:02d}.h5",
        monitor='val_loss',
        mode = 'auto',
        save_weights_only = False,
        save_freq = 'epoch',
        save_best_only=False
        ),
    
        ReduceLROnPlateau(
        monitor='val_loss',
        factor=.1,
        patience = num_reduceOnPlateu,
        verbose=1,
        min_lr=1e-7
        )
]

In [14]:
#tf.compat.v1.disable_eager_execution()

In [15]:
bottleneck = efn.EfficientNetB3(
    input_shape=input_shape_,
    weights= weights_file,
    include_top=False, 
    pooling='avg'
)
bottleneck = Model(inputs=bottleneck.inputs, outputs=bottleneck.layers[-2].output)
model = Sequential()
model.add(bottleneck)
model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dropout(.25))
model.add(Dense(512, activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(.25))
model.add(Dense(num_logits, activation='softmax', bias_initializer=output_bias))

In [16]:
if input_parameters.get('bestmodel_weights'): model.load_weights(bestmodel_weights)

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Model)                (None, 10, 10, 1536)      10783528  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
flatten (Flatten)            (None, 1536)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1536)              6144      
_________________________________________________________________
dropout (Dropout)            (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               786944    
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2

In [18]:
opt =  tf.keras.optimizers.Adam(lr=learn_rate)

In [19]:
model.compile(
    loss  = tf.keras.losses.CategoricalCrossentropy(label_smoothing = inp_label_smooth),
    #loss='categorical_crossentropy',
    optimizer=opt,
    metrics=['categorical_accuracy', kappa_keras]
)

In [20]:
history = model.fit_generator(
    train_datagen,
    steps_per_epoch=s_per_epoch,
    validation_data=val_datagen,
    validation_steps=val_steps,
    class_weight=class_weights_,
    callbacks=callbacks_list,
    epochs=num_epochs,
    verbose=1
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/2
20/20 [==============================] - 139s 7s/step - loss: 12.6282 - categorical_accuracy: 0.2400 - cohen_kappa: 0.2172 - val_loss: 1.7081 - val_categorical_accuracy: 0.2625 - val_cohen_kappa: 0.2899 - lr: 5.0000e-04
Epoch 2/2
20/20 [==============================] - 168s 8s/step - loss: 10.3958 - categorical_accuracy: 0.2900 - cohen_kappa: 0.4163 - val_loss: 1.6732 - val_categorical_accuracy: 0.3250 - val_cohen_kappa: 0.2011 - lr: 5.0000e-04


In [ ]:
dict_to_save = {}
for k, v in history.history.items():
    dict_to_save.update({k: [np.format_float_positional(x) for x in history.history[k]]})
with open(history_file, 'w') as file:
    json.dump(dict_to_save, file)


In [ ]:
model.save(model_name)
model.save_weights(file_for_weights, save_format="h5")
json_config = model.to_json()
with open(file_for_struct, 'w') as f:
    json.dump(json_config, f)

In [ ]:
plt.style.use("ggplot")
plt.figure()

plt.plot(np.arange(0, num_epochs), history.history['loss'], label='loss')
plt.plot(np.arange(0, num_epochs), history.history['val_loss'], label='val_loss')
plt.plot(np.arange(0, num_epochs),history.history['categorical_accuracy'], label='cat. accuracy')
plt.plot(np.arange(0, num_epochs),history.history['val_categorical_accuracy'], label='val_accuracy')
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(save_plot_file_main)

In [ ]:
#kappa
plt.style.use("ggplot")
plt.figure()

plt.plot(np.arange(0, num_epochs), history.history['cohen_kappa'], label='kappa')
plt.plot(np.arange(0, num_epochs), history.history['val_cohen_kappa'], label='val_kappa')

plt.title("Training Cohen Kappa on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Cohen Kappa")
plt.legend(loc="lower left")
plt.savefig(save_plot_file_kappa)

In [7]:
import os
module_name = 'effnB3_kappa'

os.system('jupyter nbconvert --to python ' + module_name + '.ipynb')
with open(module_name + '.py', 'r') as f:
    lines = f.readlines()
with open(module_name + '.py', 'w') as f:
    for line in lines:
        if 'nbconvert --to python' in line:
            break
        else:
            f.write(line)